# Count Objects in a Zone


---
<a href="https://colab.research.google.com/github/roboflow/supervision/blob/develop/docs/notebooks/count-objects-in-a-zone.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>


Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit -> Notebook settings -> Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

# Install required packages

In [2]:
!pip install -q ultralytics supervision

# Imports

In [3]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
from supervision.assets import download_assets, VideoAssets

# Download a Supervision video asset

In [ ]:
path_to_video = download_assets(VideoAssets.MARKET_SQUARE)

# Use Supervision to save the first frame to draw a zone.

In [ ]:
generator = sv.get_video_frames_generator(path_to_video)
frame = next(generator)
sv.plot_image(frame, (12, 12))

# Run Object Detection
Let's start by running the detection model on the first frame and annotating the results. In this cookbook, we use [Ultralytics](https://github.com/ultralytics/ultralytics) YOLOv8, but it can be successfully replaced with [other models](https://supervision.roboflow.com/latest/detection/core/).

We initiate the model and perform detection on the first frame of the video. Then, we convert the result into a [sv.Detections](https://supervision.roboflow.com/latest/detection/core/) object, which will be useful in the later parts of the cookbook.

In [ ]:
model = YOLO("yolov8x.pt")

results = model(frame)[0]
detections = sv.Detections.from_ultralytics(results)

The results we've obtained can be easily visualized with [sv.BoundingBoxAnnotator](https://supervision.roboflow.com/latest/annotators/#supervision.annotators.core.BoundingBoxAnnotator). By default, this annotator uses the same color to highlight objects of the same category. However, with the integration of a [tracker](https://supervision.roboflow.com/latest/trackers/), it becomes possible to assign unique colors to each tracked object. We can easily define our own [color palettes](https://supervision.roboflow.com/latest/draw/color/#supervision.draw.color.ColorPalette) and adjust parameters such as line thickness, allowing for a highly tailored visualization experience.

In [ ]:
bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=4)
annotated_frame = bounding_box_annotator.annotate(frame.copy(), detections)
sv.plot_image(annotated_frame, (12, 12))

# Improve Visualizations

Supervision annotators can be easily combined with one another. Let's enhance our visualization by adding [sv.LabelAnnotator](https://supervision.roboflow.com/latest/annotators/#supervision.annotators.core.PolygonAnnotator.annotate), which we will use to mark detections with a label indicating their category and confidence level.

In [ ]:
labels = [
    f"{results.names[class_id]} {confidence:0.2f}"
    for class_id, confidence
    in zip(detections.class_id, detections.confidence)
]

bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)
annotated_frame = frame.copy()
annotated_frame = bounding_box_annotator.annotate(annotated_frame, detections)
annotated_frame = label_annotator.annotate(annotated_frame, detections, labels)
sv.plot_image(annotated_frame, (12, 12))

# Define polygons.

We can use [VideoInfo.from_video_path](https://supervision.roboflow.com/develop/utils/video/#supervision.utils.video.VideoInfo) to learn basic information about our video, such as duration, resolution, or FPS.

In [ ]:
w, h = sv.VideoInfo.from_video_path(path_to_video).resolution_wh

With a known height and width, we can create a zone in the middle 4th of the screen. You can also use [PolygonZone](https://roboflow.github.io/polygonzone/), which enables you to draw your own polygons on an image and get the exact NumPy points. 

In [ ]:
polygon = np.array([[w/4, h/4], [w/4, 3*h/4], [3*w/4, 3*h/4], [3*w/4, h/4]], dtype=np.int32)

# Initatiate Polygon Zones and Annotators

The polygons we've created can be visualized using [sv.PolygonZone](https://supervision.roboflow.com/latest/detection/tools/polygon_zone/). This tool also allows for polygons to be displayed on the video.

The count of each people in each polygon will be displayed using [sv.PolygonZoneAnnotator](https://supervision.roboflow.com/latest/detection/tools/polygon_zone/#polygonzoneannotator).



In [ ]:
video_info = sv.VideoInfo.from_video_path(path_to_video)

zone= sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

box_annotator = sv.BoundingBoxAnnotator(thickness=4)

zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.WHITE, thickness=6, text_thickness=6, text_scale=4)

In [ ]:
#since we are only worried about the first class, which is "person"
detections = detections[detections.class_id == 0] 

#trigger determines if the detections are within the polygon zone.
zone.trigger(detections=detections)

#Annotate the frame with the updated detections for the frame
annotated_frame = zone_annotator.annotate(annotated_frame)

sv.plot_image(annotated_frame)

# Process Video

All the operations we plan to perform for each frame of our video - detection, annotation, and counting - are encapsulated in a function named callback.

In [ ]:
def callback(frame: np.ndarray, _) -> np.ndarray:
   
   #imgsz=1280 sets the image size for YOLO model input, balancing detail retention and computational efficiency
   #setting verbose equal to false will mute detection outputs of the model
   results = model(frame, imgsz=1280, verbose=False)[0] 
   detections = sv.Detections.from_ultralytics(results)

   detections = detections[detections.class_id == 0] 

   labels = [
    f"{results.names[class_id]} {confidence:0.2f}"
    for class_id, confidence
    in zip(detections.class_id, detections.confidence)
   ]

   zone.trigger(detections=detections)

   frame = bounding_box_annotator.annotate(scene=frame, detections=detections)

   frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)

   frame = zone_annotator.annotate(scene=frame)

   return frame

Finally, we are ready to process our entire video. We will use [sv.process_video](https://supervision.roboflow.com/develop/utils/video/#supervision.utils.video.process_video) and pass to it the previously defined SOURCE_VIDEO_PATH, TARGET_VIDEO_PATH, and `callback`.

In [ ]:
SOURCE_VIDEO_PATH = path_to_video
TARGET_VIDEO_PATH = "output.mp4"

In [ ]:
sv.process_video(SOURCE_VIDEO_PATH, TARGET_VIDEO_PATH, callback = callback)